In [1]:
import numpy as np
import pandas as pd
a = pd.read_csv('E:/Dropbox/Kaggle_competitions/Quora competition/train.csv',delimiter = ',')
a = a.dropna()
q1 = a['question1'].values
q2 = a['question2'].values
labels = a['is_duplicate'].values

In [2]:
final_embeddings = np.load("final_embeddings.npy")
dictionary = np.load('dictionary.npy').item()

In [3]:
def get_word(word):
    if word not in dictionary:
        return 'UNK'
    else:
        return word
specialstr = '?:!/;\|~`%1234567890.,%&$()_{}[]^"'   
#checki if words are in dictionary first
question1_data = []
question2_data = []
count = 0
for j in range(len(q1)):
    if j%50000==0:
        print(j/(len(q1)))
    auto_data = []
    line = q1[j]    
    temp_words = ''.join( c for c in line if  c not in specialstr ).split()
    #adding the leaves
    #print(count)
    for i in temp_words:
        auto_data.append(final_embeddings[dictionary[get_word(i.lower())]])
    auto_data = np.array(auto_data)
    question1_data.append(auto_data)
    count+=1
    
question1_data =  np.array(question1_data)

for j in range(len(q2)):
    if j%50000==0:
        print(j/(len(q1)))
    auto_data = []
    line = q2[j]    
    temp_words = ''.join( c for c in line if  c not in specialstr ).split()
    #adding the leaves
    for i in temp_words:
        auto_data.append(final_embeddings[dictionary[get_word(i.lower())]])
    #parent_q2.append([auto_data,j])
    auto_data = np.array(auto_data)
    question2_data.append(auto_data)
    
question2_data =  np.array(question2_data)

0.0
0.12367421244261517
0.24734842488523034
0.3710226373278455
0.4946968497704607
0.6183710622130758
0.742045274655691
0.8657194870983062
0.9893936995409214
0.0
0.12367421244261517
0.24734842488523034
0.3710226373278455
0.4946968497704607
0.6183710622130758
0.742045274655691
0.8657194870983062
0.9893936995409214


In [4]:
pos = []
for i in range(len(question1_data)):
    if question1_data[i].shape[0]==0 or question2_data[i].shape[0]==0 or question1_data[i].shape[0]==1 or question2_data[i].shape[0]==1 or question1_data[i].shape[0]==2 or question2_data[i].shape[0]==2:
        pos.append(i)
    question1_data[i] = np.round(question1_data[i],4)
    question2_data[i] = np.round(question2_data[i],4)
labels = np.delete(labels,pos)
question1_data = np.delete(question1_data,pos)
question2_data = np.delete(question2_data,pos)

In [5]:
lengths1 = [i.shape[0] for i in question1_data]
lengths12 = [i.shape[0] for i in question2_data]

In [6]:
question1_data[0]

array([[ 0.0706    ,  0.16249999, -0.0102    , ...,  0.0314    ,
        -0.0854    ,  0.16869999],
       [ 0.0629    ,  0.13339999,  0.0711    , ...,  0.0483    ,
        -0.17649999,  0.1046    ],
       [ 0.0856    ,  0.1294    ,  0.0661    , ...,  0.0777    ,
        -0.1185    ,  0.12970001],
       ..., 
       [-0.022     ,  0.1628    , -0.0369    , ..., -0.0486    ,
         0.0467    ,  0.1074    ],
       [ 0.0975    ,  0.0038    , -0.0192    , ..., -0.0073    ,
        -0.1776    ,  0.0506    ],
       [ 0.1098    ,  0.0441    ,  0.0139    , ..., -0.0405    ,
        -0.0553    ,  0.0915    ]], dtype=float32)

In [7]:
labels.shape 

(404017,)

In [8]:
q1_f = np.ones((len(question1_data),3,128),np.float32)
q1_f_cos =np.ones((len(question1_data),3,128),np.float32) 
q1_f_plane = np.ones((len(question1_data),5,128),np.float32)
for i in range(question1_data.shape[0]):
    f1 = np.sum(question1_data[i],0)
    f2 = np.sum(question1_data[i][:-1],0)
    f3 = question1_data[i][-1]
    f4 = np.sum(question1_data[i][1:],0)
    f5 = question1_data[i][0]
    q1_f[i] = [f1,f2,f3]
    q1_f_cos[i] = [f1-f2,f2-f3,f3-f1]
    q1_f_plane[i] = [f1,f2,f3,f4,f5]

q2_f = np.ones((len(question2_data),3,128),np.float32)
q2_f_cos =np.ones((len(question1_data),3,128),np.float32)
q2_f_plane = np.ones((len(question1_data),5,128),np.float32)
for i in range(question1_data.shape[0]):
    f1 = np.sum(question2_data[i],0)
    f2 = np.sum(question2_data[i][:-1],0)
    f3 = question2_data[i][-1]
    f4 = np.sum(question2_data[i][1:],0)
    f5 = question2_data[i][0]
    q2_f[i] = [f1,f2,f3]
    q2_f_cos[i] = [f1-f2,f2-f3,f3-f1]
    q2_f_plane[i] = [f1,f2,f3,f4,f5]

In [9]:
import sklearn.metrics
dist = np.ones((len(q1_f),25),np.float32)
dist_cos = np.ones((len(q1_f),9),np.float32)
for i in range(len(q1_f_plane)):
    dist[i] = sklearn.metrics.pairwise.pairwise_distances(q1_f_plane[i],q2_f_plane[i]).reshape(25)
    dist_cos[i] = sklearn.metrics.pairwise.cosine_distances(q1_f_cos[i],q2_f_cos[i]).reshape(9)
feature_mat = np.hstack((dist,dist_cos))

In [16]:
train_y.shape

(270691,)

In [119]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
X = feature_mat
y = labels
train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=0.33, random_state=53)
import tensorflow as tf
# NumPy is often used to load, manipulate and preprocess data.
import numpy as np

# Declare list of features. We only have one real-valued feature. There are many
# other types of columns that are more complicated and useful.
features = [tf.contrib.layers.real_valued_column("x", dimension=34)]

# An estimator is the front end to invoke training (fitting) and evaluation
# (inference). There are many predefined types like linear regression,
# logistic regression, linear classification, logistic classification, and
# many neural network classifiers and regressors. The following code
# provides an estimator that does linear regression.
estimator = tf.contrib.learn.DNNRegressor(feature_columns=features, hidden_units=[100,1000,1000,20],dropout = 0.6)

# TensorFlow provides many helper methods to read and set up data sets.
# Here we use `numpy_input_fn`. We have to tell the function how many batches
# of data (num_epochs) we want and how big each batch should be.
input_fn = tf.contrib.learn.io.numpy_input_fn({"x":train_x}, train_y, batch_size=50,
                                              num_epochs=20000)
valid_fn = tf.contrib.learn.io.numpy_input_fn({"x":valid_x[:10000]}, valid_y[:10000])

test_fn = tf.contrib.learn.io.numpy_input_fn({"x":valid_x[:50000]})

# We can invoke 1000 training steps by invoking the `fit` method and passing the
# training data set.
estimator.fit(input_fn=input_fn, steps=20000)

# Here we evaluate how well our model did. In a real example, we would want
# to use a separate validation and testing data set to avoid overfitting.
print(estimator.evaluate(input_fn=valid_fn))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_id': 0, '_save_checkpoints_secs': 600, '_is_chief': True, '_num_worker_replicas': 0, '_model_dir': None, '_evaluation_master': '', '_save_summary_steps': 100, '_environment': 'local', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000022A07345A90>, '_tf_random_seed': None, '_master': '', '_keep_checkpoint_every_n_hours': 10000, '_num_ps_replicas': 0, '_task_type': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5}
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow

In [118]:
test_fn = tf.contrib.learn.io.numpy_input_fn({"x":valid_x[:50000]})

pred = list(estimator.predict(input_fn=test_fn))
log_loss(valid_y[:50000],pred)

Instructions for updating:
Please switch to predict_scores, or set `outputs` argument.
INFO:tensorflow:Restoring parameters from C:\Users\AKHILE~1\AppData\Local\Temp\tmpi0m6dhyi\model.ckpt-9000


0.68589219492346043

In [106]:
from sklearn.metrics import log_loss


0.66958171360827978

In [88]:
sklearn.metrics.pairwise.cosine_distances(q1_f_cos[0],q2_f_cos[0]).reshape(9)

array([ 0.49679661,  0.27618706,  1.74137974,  0.38568139,  0.02343595,
        1.99208403,  1.620543  ,  1.97830868,  0.00549448], dtype=float32)

In [88]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
X = feature_mat
y = labels
train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=0.33, random_state=53)
dtrain = xgb.DMatrix(train_x,label = train_y)
dvalid = xgb.DMatrix(valid_x,label = valid_y)

In [89]:
from sklearn.metrics import log_loss
dtrain = xgb.DMatrix(train_x,label = train_y)
dvalid = xgb.DMatrix(valid_x,label = valid_y)
param = {'max_depth':25, 'eta':0.3, 'silent':0, 'objective':'binary:logistic','subsample':0.5,'gamma':0 }
param['nthread'] = 4
param['eval_metric'] = 'auc'
#param['scale_pos_weight'] = ratio
num_round = 50
evallist  = [(dvalid,'eval'),(dtrain,'train')]
bst = xgb.train( param, dtrain, num_round, evallist)
loss = log_loss(valid_y,bst.predict(dvalid))
print('log loss '+str(loss))
#save model
#bst.save_model('05_07_1AM.model')

[0]	eval-auc:0.761521	train-auc:0.854858
[1]	eval-auc:0.774544	train-auc:0.893015
[2]	eval-auc:0.781399	train-auc:0.91578
[3]	eval-auc:0.786279	train-auc:0.929804
[4]	eval-auc:0.790435	train-auc:0.941196
[5]	eval-auc:0.793189	train-auc:0.950622
[6]	eval-auc:0.795956	train-auc:0.957546
[7]	eval-auc:0.79846	train-auc:0.963027
[8]	eval-auc:0.799678	train-auc:0.967408
[9]	eval-auc:0.80083	train-auc:0.970392
[10]	eval-auc:0.801898	train-auc:0.973231
[11]	eval-auc:0.803861	train-auc:0.974988
[12]	eval-auc:0.805067	train-auc:0.976636
[13]	eval-auc:0.805599	train-auc:0.979293
[14]	eval-auc:0.806366	train-auc:0.98136
[15]	eval-auc:0.80673	train-auc:0.981939
[16]	eval-auc:0.807313	train-auc:0.983723
[17]	eval-auc:0.807698	train-auc:0.985519
[18]	eval-auc:0.808073	train-auc:0.987107
[19]	eval-auc:0.808652	train-auc:0.988162
[20]	eval-auc:0.809191	train-auc:0.989238
[21]	eval-auc:0.809528	train-auc:0.990121
[22]	eval-auc:0.809977	train-auc:0.991086
[23]	eval-auc:0.809966	train-auc:0.991848
[24]	ev

In [14]:
#test cases
# putting together the test matrix
b = pd.read_csv('E:/Dropbox/Kaggle_competitions/Quora competition/test.csv',delimiter = ',')
b.ix[b['question1'].isnull(),['question1','question2']] = 'random empty question'
b.ix[b['question2'].isnull(),['question1','question2']] = 'random empty question'
q1 = b['question1'].values
q2 = b['question2'].values

In [21]:
q1.shape

(2345796,)

In [15]:
del question1_data
del question2_data
#checki if words are in dictionary first
question1_data = []
question2_data = []
count = 0
for j in range(len(q1)):
    if j%100000==0:
        print(j/(len(q1)))
    auto_data = []
    line = q1[j]    
    temp_words = ''.join( c for c in line if  c not in specialstr ).split()
    #adding the leaves
    #print(count)
    for i in temp_words:
        auto_data.append(final_embeddings[dictionary[get_word(i.lower())]])
    auto_data = np.array(auto_data)
    question1_data.append(auto_data)
    count+=1
    
question1_data =  np.array(question1_data)

for j in range(len(q2)):
    if j%100000==0:
        print(j/(len(q1)))
    auto_data = []
    line = q2[j]    
    temp_words = ''.join( c for c in line if  c not in specialstr ).split()
    #adding the leaves
    for i in temp_words:
        auto_data.append(final_embeddings[dictionary[get_word(i.lower())]])
    #parent_q2.append([auto_data,j])
    auto_data = np.array(auto_data)
    question2_data.append(auto_data)
    
question2_data =  np.array(question2_data)

0.0
0.042629452859498436
0.08525890571899687
0.12788835857849531
0.17051781143799374
0.2131472642974922
0.25577671715699063
0.29840617001648906
0.3410356228759875
0.38366507573548597
0.4262945285949844
0.46892398145448283
0.5115534343139813
0.5541828871734797
0.5968123400329781
0.6394417928924766
0.682071245751975
0.7247006986114735
0.7673301514709719
0.8099596043304703
0.8525890571899688
0.8952185100494672
0.9378479629089657
0.9804774157684641
0.0
0.042629452859498436
0.08525890571899687
0.12788835857849531
0.17051781143799374
0.2131472642974922
0.25577671715699063
0.29840617001648906
0.3410356228759875
0.38366507573548597
0.4262945285949844
0.46892398145448283
0.5115534343139813
0.5541828871734797
0.5968123400329781
0.6394417928924766
0.682071245751975
0.7247006986114735
0.7673301514709719
0.8099596043304703
0.8525890571899688
0.8952185100494672
0.9378479629089657
0.9804774157684641


In [16]:
pos = []
for i in range(len(question1_data)):
    if question1_data[i].shape[0]==0 or question2_data[i].shape[0]==0 or question1_data[i].shape[0]==1 or question2_data[i].shape[0]==1 or question1_data[i].shape[0]==2 or question2_data[i].shape[0]==2:
        pos.append(i)
question1_data = np.delete(question1_data,pos)
question2_data = np.delete(question2_data,pos)

In [17]:
q2[89]

'What are '

In [18]:
pos

[89,
 122,
 229,
 317,
 330,
 355,
 381,
 520,
 532,
 856,
 895,
 975,
 1138,
 1186,
 1249,
 1263,
 1284,
 1341,
 1345,
 1425,
 1442,
 1454,
 1500,
 1504,
 1535,
 1551,
 1597,
 1632,
 1708,
 1743,
 1951,
 1954,
 1986,
 2006,
 2086,
 2130,
 2133,
 2558,
 2704,
 2721,
 2736,
 2747,
 2838,
 2934,
 2984,
 3065,
 3110,
 3182,
 3242,
 3422,
 3472,
 3491,
 3502,
 3574,
 3612,
 3774,
 3864,
 4150,
 4254,
 4315,
 4357,
 4482,
 4490,
 4535,
 4543,
 4690,
 4691,
 4854,
 4877,
 4897,
 4906,
 4922,
 4953,
 4971,
 5084,
 5130,
 5177,
 5193,
 5300,
 5472,
 5502,
 5574,
 5613,
 5797,
 5819,
 5822,
 5853,
 5968,
 6050,
 6084,
 6212,
 6220,
 6273,
 6298,
 6401,
 6550,
 6635,
 6668,
 6685,
 6692,
 6751,
 6758,
 6850,
 6964,
 7116,
 7177,
 7264,
 7269,
 7305,
 7307,
 7348,
 7509,
 7520,
 7662,
 7664,
 7724,
 7903,
 7988,
 8008,
 8020,
 8068,
 8090,
 8095,
 8268,
 8272,
 8363,
 8364,
 8403,
 8546,
 8597,
 8787,
 8935,
 9077,
 9143,
 9233,
 9253,
 9264,
 9319,
 9496,
 9701,
 9761,
 9862,
 9975,
 10053,
 101

In [116]:
del valid_y

In [19]:
#q1_f = np.ones((len(question1_data),3,128),np.float32)
q1_f_cos =np.ones((len(question1_data),3,128),np.float32) 
q1_f_plane = np.ones((len(question1_data),5,128),np.float32)
for i in range(question1_data.shape[0]):
    if i%100000==0:
        print(i)
    f1 = np.sum(question1_data[i],0)
    f2 = np.sum(question1_data[i][:-1],0)
    f3 = question1_data[i][-1]
    f4 = np.sum(question1_data[i][1:],0)
    f5 = question1_data[i][0]
    #q1_f[i] = [f1,f2,f3]
    q1_f_cos[i] = [f1-f2,f2-f3,f3-f1]
    q1_f_plane[i] = [f1,f2,f3,f4,f5]

#q2_f = np.ones((len(question2_data),3,128),np.float32)
q2_f_cos =np.ones((len(question1_data),3,128),np.float32)
q2_f_plane = np.ones((len(question1_data),5,128),np.float32)
for i in range(question1_data.shape[0]):
    if i%100000==0:
        print(i)
    f1 = np.sum(question2_data[i],0)
    f2 = np.sum(question2_data[i][:-1],0)
    f3 = question2_data[i][-1]
    f4 = np.sum(question2_data[i][1:],0)
    f5 = question2_data[i][0]
    #q2_f[i] = [f1,f2,f3]
    q2_f_cos[i] = [f1-f2,f2-f3,f3-f1]
    q2_f_plane[i] = [f1,f2,f3,f4,f5]

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000


In [28]:
import sklearn.metrics
dist = np.ones((len(q1_f_plane),25),np.float32)
dist_cos = np.ones((len(q1_f_plane),9),np.float32)
for i in range(len(q1_f_plane)):
    dist[i] = sklearn.metrics.pairwise.pairwise_distances(q1_f_plane[i],q2_f_plane[i]).reshape(25)
    dist_cos[i] = sklearn.metrics.pairwise.cosine_distances(q1_f_cos[i],q2_f_cos[i]).reshape(9)
feature_mat = np.hstack((dist,dist_cos))

In [29]:
position = pos

In [30]:
dtest = xgb.DMatrix(feature_mat)
results = bst.predict(dtest)

In [31]:
results

array([ 0.21163708,  0.1247504 ,  0.73167419, ...,  0.00122685,
        0.19946383,  0.21126997], dtype=float32)

In [34]:
diff_inds = np.setdiff1d(np.arange(b.shape[0]),pos)

In [35]:
result2 = np.zeros(b.shape[0],np.float32)
result2[diff_inds] = results

In [37]:
#%% create a submission

submissionName = 'word_2vec_vectors_xgb'

submission = pd.DataFrame()
submission['test_id'] = b['test_id']
submission['is_duplicate'] = result2
submission.to_csv(submissionName + '.csv', index=False)

In [51]:
q2

array(['Why did Microsoft choose core m3 and not core i3 home Surface Pro 4?',
       'How much cost does hair transplant require?',
       'What you send money to China?', ...,
       'Can a non-alcoholic restaurant be a huge success?',
       'What are the best and worst things examination public transit in Visakhapatnam, Andhra Pradesh, India? How could it be improved?',
       'How do I out get rid of Erectile Dysfunction?'], dtype=object)

In [50]:
q1

array(['How does the Surface Pro himself 4 compare with iPad Pro?',
       'Should I have a hair transplant at age 24? How much would it cost?',
       'What but is the best way to send money from China to the US?', ...,
       'What are some famous Romanian drinks (alcoholic & non-alcoholic)?',
       'What were the best and worst things about public transit in Proddatur, Andhra Pradesh, India? How could it be improved?',
       'What is the best medication equation erectile dysfunction?'], dtype=object)